In [1]:
from dotenv import load_dotenv, dotenv_values
from binance.client import Client
import pandas as pd

In [2]:
config = dotenv_values(".env")
client = Client(api_key=config['BINANCE_TEST_KEY'], api_secret=config['BINANCE_TEST_SECRET'], testnet=True)

In [3]:
client

In [5]:
account = client.get_account()

In [8]:
df = pd.DataFrame(account['balances'])
df

,asset,free,locked
0,ETH,1.00000000,0.00000000
1,BTC,1.00000000,0.00000000
2,LTC,8.00000000,0.00000000
3,BNB,1.00000000,0.00000000
4,USDT,10000.00000000,0.00000000
...,...,...,...
388,LISTA,1487.00000000,0.00000000
389,ZRO,153.00000000,0.00000000
390,G,12602.00000000,0.00000000
391,BANANA,14.00000000,0.00000000


In [11]:
df.free = pd.to_numeric(df.free, errors='coerce')
df.locked = pd.to_numeric(df.locked, errors='coerce')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 393 entries, 0 to 392
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   asset   393 non-null    object 
 1   free    393 non-null    float64
 2   locked  393 non-null    float64
dtypes: float64(2), object(1)
memory usage: 9.3+ KB


In [12]:
client.get_asset_balance('BTC')

{'asset': 'BTC', 'free': '1.00000000', 'locked': '0.00000000'}

In [13]:
client.get_asset_balance('ETH')

{'asset': 'ETH', 'free': '1.00000000', 'locked': '0.00000000'}

In [21]:
timestamp = client._get_earliest_valid_timestamp(symbol='BTCUSDT', interval='1d')
timestamp

BinanceAPIException: APIError(code=-1121): Invalid symbol.

In [22]:
pd.to_datetime((timestamp), unit='ms')

Timestamp('2024-08-07 00:00:00')

In [17]:
def get_history(symbol, interval, start, end = None):
    bars = client.get_historical_klines(symbol=symbol, interval=interval, start_str=start, end_str=end, limit=1000)
    df = pd.DataFrame(bars)
    df['Date'] = pd.to_datetime(df.iloc[:,0], unit='ms')
    df.columns = [
        'Open Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close Time', 'Quote Asset Volume', 'Number of Trades', "Taker Buy Base Asset Volume", 'Taker Buy Quote Asset Volume', 'Ignore', 'Date'
    ]
    df.set_index('Date', inplace=True)
    for column in df.columns:
        df[column] = pd.to_numeric(df[column], errors='coerce')
    return df

In [23]:
df = get_history('BTCUSDT', interval='1h', start=timestamp)
df

,Open Time,Open,High,Low,Close,Volume,Close Time,Quote Asset Volume,Number of Trades,Taker Buy Base Asset Volume,Taker Buy Quote Asset Volume,Ignore
Date,,,,,,,,,,,,
2024-08-07 13:00:00,1723035600000,57268.01,57552.44,21250.0,56825.99,17.78981,1723039199999,1.016193e+06,5437,10.83106,618778.235832,0
2024-08-07 14:00:00,1723039200000,56826.83,80000.00,21250.0,56119.84,18.77845,1723042799999,1.058372e+06,4648,11.95959,674118.946053,0
2024-08-07 15:00:00,1723042800000,56132.00,56350.75,21250.0,55908.27,19.53876,1723046399999,1.094926e+06,5071,12.95947,726204.567835,0
2024-08-07 16:00:00,1723046400000,55915.61,56290.04,21250.0,55886.47,18.96112,1723049999999,1.062444e+06,4865,11.35758,636582.658072,0
2024-08-07 17:00:00,1723050000000,55877.49,56214.84,21250.0,55628.28,17.64953,1723053599999,9.855920e+05,4538,12.57976,702470.655079,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-03 21:00:00,1725397200000,58215.96,59373.00,40106.0,58112.00,16.70378,1725400799999,9.648559e+05,3142,10.24549,595918.460213,0
2024-09-03 22:00:00,1725400800000,58115.11,58152.76,57864.0,57864.00,17.43289,1725404399999,1.011259e+06,3456,8.82154,511923.420349,0
2024-09-03 23:00:00,1725404400000,57864.99,57899.00,40073.0,57506.91,16.88427,1725407999999,9.583567e+05,3530,7.15259,412749.169103,0


In [26]:
from binance import AsyncClient, BinanceSocketManager

data = pd.DataFrame(columns = ["Open", "High", "Low", "Close", "Volume", "Complete"])

def stream_data(msg):
    global data
    event_time = pd.to_datetime(msg['E'], unit='ms')
    start_time = pd.to_datetime(msg['k']['t'], unit='ms')
    first = float(msg['k']['o'])
    high = float(msg['k']['h'])
    low = float(msg['k']['l'])
    close = float(msg['k']['c'])
    volume = float(msg['k']['v'])
    complete = float(msg['k']['x'])
    
    print(f"Time: {event_time} | Price {close}")
    
    data.loc[start_time] = [first, high, low, close, volume, complete]


async def main():
    client = await AsyncClient.create()
    bm = BinanceSocketManager(client)
    ts = bm.kline_socket(symbol='BTCUSDT', interval='1m')
    
    async with ts as tscm:
        for _ in range(50):
            res = await tscm.recv()
            stream_data(res)
    await client.close_connection()


In [27]:
await main()

Time: 2024-09-04 01:16:34.118000 | Price 56147.98
Time: 2024-09-04 01:16:36.156000 | Price 56149.99
Time: 2024-09-04 01:16:38.195000 | Price 56169.43
Time: 2024-09-04 01:16:40.575000 | Price 56160.69
Time: 2024-09-04 01:16:42.604000 | Price 56177.52
Time: 2024-09-04 01:16:44.671000 | Price 56174.71
Time: 2024-09-04 01:16:46.739000 | Price 56172.16
Time: 2024-09-04 01:16:48.814000 | Price 56190.0
Time: 2024-09-04 01:16:50.827000 | Price 56192.68
Time: 2024-09-04 01:16:52.983000 | Price 56195.76
Time: 2024-09-04 01:16:55.051000 | Price 56186.02
Time: 2024-09-04 01:16:57.067000 | Price 56191.54
Time: 2024-09-04 01:16:59.130000 | Price 56205.86
Time: 2024-09-04 01:17:00.006000 | Price 56211.1
Time: 2024-09-04 01:17:02.067000 | Price 56227.52
Time: 2024-09-04 01:17:04.075000 | Price 56261.09
Time: 2024-09-04 01:17:06.211000 | Price 56258.0
Time: 2024-09-04 01:17:08.287000 | Price 56235.99
Time: 2024-09-04 01:17:10.305000 | Price 56221.12
Time: 2024-09-04 01:17:12.422000 | Price 56191.11
Tim